# Phase correlations

In this notebook we play around with linear phase correlations of white noise.

In [1]:
from plotly import tools
from plotly import offline as py
from plotly import graph_objs as go
from plotly import figure_factory as ff

py.init_notebook_mode(connected=True)

In order to create linear phase correlated white noise we start with normal white noise $(x_1,\dots,x_n)$. The real fourier transform is given through $(X_0,\dots,X_{n/2})$. $X_0$ equals the signal mean and should not be modified. We can write $X_i=|X_i|e^{i\phi_i}$. Now we apply the transform,
$$\phi_i=\phi_{i+j}+\phi+\eta_i$$,
wherein $phi\in[-\pi,+\pi[$ is a constant, $\eta_i$ a noise term. With regard to the index shift $i+j$ we apply the circular condition $i+j = i+j-n/2$ if $i+j>n/2-1$.

In [84]:
def correlate_phases(x, offset, phase, noise):
    X = np.fft.rfft(x)
    
    phi = np.arctan2(X.imag, X.real)
    phi[1:] = np.roll(phi[1:], -offset) + phase + noise
    
    X = np.abs(X) * np.exp(1j * phi)
    
    return np.fft.irfft(X)

In [85]:
x = np.random.normal(0, 1, 100)

n = np.random.normal(0, 0.5, 50)

y1 = correlate_phases(x, 1, 0, n)
y2 = correlate_phases(x, 1, np.pi, n)

In [86]:
layout = go.Layout(
    legend=dict(orientation='h')
)

figure = go.Figure([
    go.Scatter(y=x, name='white noise'),
    go.Scatter(y=y1, name='phase correlated (δ=0)'),
#    go.Scatter(y=y2, name='phase correlated (δ=π)'),    
], layout)

py.iplot(figure)

If we apply the linear phase correlation without constant phase offset. We find that there are areas where the signal has large overlap with the original uncorrelated white noise.

In [87]:
layout = go.Layout(
    legend=dict(orientation='h')
)

figure = go.Figure([
    go.Scatter(y=x, name='white noise'),
    go.Scatter(y=y2, name='phase correlated (δ=π)'),    
], layout)

py.iplot(figure)

When using a constant phase offset to obtain the linear phase correlated white noise the local similarity is reduced compared to the previous time series. However, global structural features are still similar.

In [96]:
z1 = correlate_phases(x, 1, 0, n)
z2 = correlate_phases(x, 2, 0, n)

In [97]:
layout = go.Layout(
    legend=dict(orientation='h')
)

figure = go.Figure([
    go.Scatter(y=x, name='white noise'),
    go.Scatter(y=z1, name='phase correlated (i=1)'),
    go.Scatter(y=z2, name='phase correlated (i=2)'),    
], layout)

py.iplot(figure)